In [1]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.1/777.1 MB 2.2 MB/s eta 0:00:0000:0100:03
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [36]:
!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 15.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import json
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import string
import nltk
from nltk.corpus import stopwords
import spacy
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter 
import statsmodels.api as sm
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from numpy import array
from sklearn.decomposition import TruncatedSVD

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/LJL/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
def tokenize(sentence):
    sentence = re.sub(r"(?:\@|https?\://)\S+|\n+", "", sentence.lower())
    return sentence

In [39]:
data = pd.read_csv("../buffet_reviews.csv")
result = [tokenize(x) for x in data.text]
nltk.download('vader_lexicon')
nlp = spacy.load("en_core_web_lg")
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/LJL/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [8]:
def find_sentiment(doc):
    # find roots of all entities in the text
    ner_heads = {ent.root.idx: ent for ent in doc.ents}
    rule3_pairs = []
    for token in doc:
        children = token.children
        A = "999999"
        M = "999999"
        add_neg_pfx = False
        for child in children:
            if(child.dep_ == "nsubj" and not child.is_stop): # nsubj is nominal subject
                if child.idx in ner_heads:
                    A = ner_heads[child.idx].text
                else:
                    A = child.text
            if(child.dep_ == "acomp" and not child.is_stop): # acomp is adjectival complement
                M = child.text
            # example - 'this could have been better' -> (this, not better)
            if(child.dep_ == "aux" and child.tag_ == "MD"): # MD is modal auxiliary
                neg_prefix = "not"
                add_neg_pfx = True
            if(child.dep_ == "neg"): # neg is negation
                neg_prefix = child.text
                add_neg_pfx = True
        if (add_neg_pfx and M != "999999"):
            M = neg_prefix + " " + M
        if(A != "999999" and M != "999999"):
            rule3_pairs.append((A, M, sid.polarity_scores(M)['compound']))
    return rule3_pairs

In [9]:
aspect_opinion = [find_sentiment(nlp(x)) for x in result]
aspect_opinion

KeyboardInterrupt: 

In [13]:
for i in range(len(data)):
    biz_id = data.loc[data.index[i], "business_id"]
    text = data.loc[data.index[i], "text"]
    cleaned = tokenize(text)
    print(cleaned)
    things = find_sentiment(nlp(cleaned))
    print(things)
    

spectacular, one of a kind restaurant with an elegant, serene atmosphere. they have the best deals during september, as you can order the magical dining all you can eat option for about half off the original price. one of my favorite places for exquisitely made steak and simple but delicious desserts.
[]
excellent service,food,atmosphere, this is definitely one of the best restaurants in orlando but a bit pricey but still worth every penny. reservations suggested if not u will wait for a long time. food quality- (out of 5 stars) 4atmosphere ( out of 5 stars) 5service ( out of 5 stars) 4
[]
delicious!  amazing salad bar, wonderful meat.  come here with a good appetite!  too bad they don't have one in socal.
[]
it's really nice a lot of food there has to be something for everyone best chinese buffet in orlando
[]
i think the amount of fun you will have here depends on who you go with and what you want to do. i went with my son today (2/23) on a class trip and he was very disappointed. fa

[('service', 'impeccable', 0.0)]
everything is delicious! the salad bar is great, everything tastes fresh. but don't fill up too much because the meat is the best part. my favorites are the flank steak and the house sirloin. and they have my favorite dessert in the world; the bananas foster pie! soooo good!!!
[('bar', 'great', 0.6249)]
for asian buffets, this stands a bit above the rest of the field in terms of quality and selection.  this buffet has mainly chinese and japanese items.  they have sushi freshly made and a hibachi grill chef that takes your order that is included with the buffet, which i thought was very well priced at $12 for adult.  i had hibachi shrimp and scallops with fried rice (hibachi cooked).  it was very tasty, but the scallops were about the size of a hot cocoa mini marshmallow.  i did not know they harvested scallops that small!!  i did not have to wait very long, but can see how there could be a wait if it is busy as there was only 1 hibachi chef.i also had t

[('bar', 'impressive', 0.5106), ('décor', 'elegant', 0.4767)]
standard japanese/chinese buffet.  very reasonable prices - especially if you use a coupon from yelp or in the coupon books that are ubiquitous in orlando.large sushi station with a surprising selection.  the sushi is obviously frozen and thawed out.  if you don't have high expectations this area may be a pleasant surprise.hot areas have chinese and japanese food but it is overwhelmingly dominated by the former.  it is owned and operated by chinese people.  the food was average.  in the early evening the food turnover was fairly quick.  you need to pay extra if you want the crab legs.the salad bar was weak and desserts weren't too bad.supposedly one of the better asian buffets in the international drive area...
[('food', 'average', 0.0), ('turnover', 'quick', 0.0), ('bar', 'weak', -0.4404)]
i wish i found this sooner than another buffet i tried out. it's so dirt cheap with amazing selection that you literally feel like you'r

KeyboardInterrupt: 

In [46]:
from textblob import TextBlob
from tqdm import tqdm

aspects = []
with tqdm(total=len(data)) as pbar:
    for i in range(len(data)):
        biz_id = data.loc[data.index[i], "business_id"]
        text = data.loc[data.index[i], "text"]
        review_id = data.loc[data.index[i], "review_id"]
        doc = nlp(text)
        descriptive_term = ''
        target = ''
        for token in doc:
            if token.dep_ != 'nsubj' and token.pos_ != 'NOUN':
                continue
            for child in token.children:
                if child.pos_ == "ADJ" or child.pos_ == "ADV":
                    aspects.append([biz_id, review_id, token.text, child.text, 
                                    TextBlob(child.text).sentiment.polarity])
        pbar.update()
print(aspects)

100%|██████████████████████████████████| 17719/17719 [14:04<00:00, 20.99it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




In [48]:
aspects_df = pd.DataFrame(data=aspects, columns=["business_id", "review_id",
                "aspect", "description", "polarity"])
aspects_df

,business_id,review_id,aspect,description,polarity
0,5K6W5eZKzJNImYVxHG7G4g,CkP9x1pgLwGn7xtF4jz6-A,restaurant,kind,0.6
1,5K6W5eZKzJNImYVxHG7G4g,CkP9x1pgLwGn7xtF4jz6-A,atmosphere,elegant,0.5
2,5K6W5eZKzJNImYVxHG7G4g,CkP9x1pgLwGn7xtF4jz6-A,atmosphere,serene,0.0
3,5K6W5eZKzJNImYVxHG7G4g,CkP9x1pgLwGn7xtF4jz6-A,deals,best,1.0
4,5K6W5eZKzJNImYVxHG7G4g,CkP9x1pgLwGn7xtF4jz6-A,half,about,0.0
...,...,...,...,...,...
101832,6-nX4fFhEBZBOGPZGGl5ng,dG3kfEG_mUs4RxIV9Ud10Q,service,ever,0.0
101833,6-nX4fFhEBZBOGPZGGl5ng,dG3kfEG_mUs4RxIV9Ud10Q,attitude,bad,-0.7
101834,6-nX4fFhEBZBOGPZGGl5ng,dG3kfEG_mUs4RxIV9Ud10Q,education,rude,-0.3
101835,6-nX4fFhEBZBOGPZGGl5ng,dG3kfEG_mUs4RxIV9Ud10Q,education,bad,-0.7


In [49]:
aspects_df.to_csv("../buffet_review_aspect.csv",index=False)

In [53]:
list(aspects_df["aspect"].value_counts().index)

['food',
 'time',
 'buffet',
 'place',
 'experience',
 'service',
 'restaurant',
 'thing',
 'options',
 'selection',
 'restaurants',
 'items',
 'dishes',
 'variety',
 'times',
 'meal',
 'things',
 'price',
 'day',
 'buffets',
 'people',
 'pot',
 'places',
 'bar',
 'quality',
 'part',
 'staff',
 'way',
 'choices',
 'meat',
 'potatoes',
 'foods',
 'night',
 'visit',
 'reviews',
 'side',
 'breakfast',
 'deal',
 'meats',
 'amount',
 'atmosphere',
 'ones',
 'sushi',
 'sauce',
 'tables',
 'value',
 'flavor',
 'dinner',
 'area',
 'rice',
 'chicken',
 'one',
 'rib',
 'plates',
 'reason',
 'table',
 'cuisine',
 'rolls',
 'trip',
 'selections',
 'spot',
 'bit',
 'characters',
 'dish',
 'years',
 'minutes',
 'option',
 'taste',
 'stuff',
 'desserts',
 'job',
 'fare',
 'plate',
 'location',
 'salad',
 'choice',
 'fun',
 'types',
 'family',
 'park',
 'kids',
 'bread',
 'drinks',
 'lunch',
 'beans',
 'hour',
 'menu',
 'show',
 'server',
 'fan',
 'style',
 'line',
 'prices',
 'room',
 'fish',
 'salad